In [1]:
from utils.data_utils import *
import unsupervised.model as unsupervised_model
from torch import nn,optim
from torch.utils.data import  DataLoader
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 32
batch_size = 2048
input_size = 1
window_size = 10
num_layers = 2
hidden_size = 64

In [3]:
logPath = 'D:\\毕业设计\\loghub\\HDFS_1\\HDFS.log'
parsed_result = '.\\data\\parser\\log_item_to_label.csv'
cluster_result = '.\\data\\parser\\'
# partition
partition_output = '.\\data\\lstm\\dataset_official\\'
instance_file = 'instance.csv'
label_file = '.\\data\\partition\\anormaly_label.csv'
normal_output = 'normal.csv'
abnormal_output = 'abnormal.csv'

#lstm dataset
lstm_dataset = '.\\data\\lstm\\dataset_official\\'
unsupervised_train = 'train.csv'
unsupervised_test_normal = 'normal.csv'
unsupervised_test_abnormal = 'abnormal.csv'

predicted_input = 'unsupervised_input.csv'
predicted_normal_output = 'unsupervised_normal_output.csv'
predicted_abnormal_output = 'unsupervised_abnormal_output.csv'
supervised_test = 'supervised_test.csv'

In [4]:
model_dir = 'model'
input_size = 1
model_usl_name = 'unsupervise_lstm_SGD'
# model_usl_name = 'type={}_version={}'.format('official_unsupervised',version)

In [5]:
def generate_predicted_data(name, window_size=10):

    event_seq_set = set()
    data = pd.read_csv(name, engine='c', na_filter=False, memory_map=True)
    # blockId_list= data['BlockId'].tolist()
    event_sequence = data['EventSequence'].apply(literal_eval).tolist()
    block_to_seq = []
    for i,line in enumerate(event_sequence[:4000]):
        if tuple(line) not in event_seq_set:
            block_to_seq.append([i,line])
            event_seq_set.add(tuple(line))

    session_to_seq = []
    seqs = []
    labels = []
    seq_count = 0
    event_seq_set = list(event_seq_set)
    for line in tqdm(block_to_seq, name):
        session = []
        line = line[1]
        line = [0]+list(line)+[30]
        line = line + [-1] * (window_size + 1 - len(line))
        for i in range(len(line) - window_size):
            seq = line[i:i + window_size]
            label = line[i + window_size]
            seqs.append(seq)
            session.append(seq_count)
            labels.append(label)
            seq_count += 1
        session_to_seq.append(session)
    print('Number of sessions({}): {}'.format(name, len(session_to_seq)))
    print('Number of seqs({}): {}'.format(name, len(seqs)))
    dataset = TensorDataset(torch.tensor(seqs, dtype=torch.float), torch.tensor(labels))
    return session_to_seq, dataset, block_to_seq


In [5]:
train_dataset = generate_train_data(lstm_dataset + unsupervised_train)
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
batch_size = 20000
test_normal_session, test_normal_dataset, normal_block = generate_predicted_data(
    lstm_dataset + 'no_dup_normal.csv', window_size)
# normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_abnormal_session, test_abnormal_dataset, abnormal_block = generate_predicted_data(
    lstm_dataset + 'no_dup_abnormal.csv', window_size)
abnormal_dataloader = DataLoader(test_abnormal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

loading data: 100%|█████████████████████████████████████████████████████████████| 4855/4855 [00:00<00:00, 25832.24it/s]


Number of sessions(.\data\lstm\dataset_official\train.csv): 4855
Number of seqs(.\data\lstm\dataset_official\train.csv): 99980


.\data\lstm\dataset_official\no_dup_normal.csv: 100%|█████████████████████████| 14177/14177 [00:01<00:00, 11154.11it/s]


Number of sessions(.\data\lstm\dataset_official\no_dup_normal.csv): 14177
Number of seqs(.\data\lstm\dataset_official\no_dup_normal.csv): 269570


.\data\lstm\dataset_official\no_dup_abnormal.csv: 100%|█████████████████████████| 4123/4123 [00:00<00:00, 11583.29it/s]


Number of sessions(.\data\lstm\dataset_official\no_dup_abnormal.csv): 4123
Number of seqs(.\data\lstm\dataset_official\no_dup_abnormal.csv): 88410


In [6]:
# test_normal_session = test_normal_session[:4100]
# test_normal_dataset = test_normal_dataset[]
normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [7]:
model = unsupervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=31)
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9)

In [8]:
writer = SummaryWriter(log_dir='log/' + model_usl_name)

In [9]:
current_epoch = 0
num_epochs = 400

In [10]:
def predict(model, dataloader, sessions, num_candidates=5, window_size=10, ts=0.0003):
    softmax = nn.Softmax(dim=1)
    # Test the model
    result = []
    seq_result = []
    with torch.no_grad():
        for step, (seq, labels) in tqdm(enumerate(dataloader), desc='loading data'):
            seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
            output = model(seq).cpu()
            output = output[:, -1, :]
            prob = softmax(output)
            predicted = torch.argsort(output, 1)[:, -num_candidates:]
            for i, label in enumerate(labels):
                if label not in predicted[i] or prob[i][label] < ts:
                    seq_result.append(True)
                else:
                    seq_result.append(False)

    for session in sessions:
        positive = False
        for seq_id in session:
            if seq_result[seq_id] == True:
                positive = True
                break
        result.append(positive)

    return result

# count_metries
def count_metries(model, normal_dataloader, abnormal_dataloader, test_normal_session, test_abnormal_session, num_candidates=5, window_size=10, ts=0.0003):

    model.eval()
    # Test the model
    start_time = time.time()
    test_normal_result = predict(model,normal_dataloader,test_normal_session,num_candidates,window_size,ts)
    test_abnormal_result = predict(model,abnormal_dataloader,test_abnormal_session,num_candidates,window_size,ts)
    elapsed_time = time.time() - start_time
    print('elapsed_time: {:.3f}s'.format(elapsed_time))
    TP = test_abnormal_result.count(True)
    FP = test_normal_result.count(True)
    TN = len(test_normal_session) - FP
    # Compute precision, recall and F1-measure
    FN = len(test_abnormal_session) - TP
    A = 100 * (TP+TN)/(TP+FP+FN+TN)
    P = 100 * TP / (TP + FP)
    R = 100 * TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
    print('Finished Predicting')
    return A,P,R,F1

In [11]:
def metric(steps):
    model.eval()
    A,P,R,F1 = count_metries(model, normal_dataloader, abnormal_dataloader,
                                test_normal_session, test_abnormal_session,
                                10, window_size,ts=0)
    result.append([steps,[A,P,R,F1]])
    print("正确率:{} 准确率:{} 召回率:{} F1值:{}".format(round(A,3),round(P,3),round(R,3),round(F1,3)))
    writer.add_scalar('正确率', A, steps)
    writer.add_scalar('准确率', P, steps)
    writer.add_scalar('召回率', R, steps)
    writer.add_scalar('F1值', F1, steps)
    model.train()

In [12]:
total_step = len(dataloader)
print(total_step)
model = unsupervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=31)
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9)

49


In [ ]:
total_step = len(dataloader)
start_time = time.time()
result = []
steps = 0
for epoch in range(current_epoch,num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    model.train()
    for step, (seq, label) in enumerate(dataloader):
        # Forward pass
        if epoch<2 and steps%10==0:
            metric(steps)
        steps+=1
        seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
        label1= seq[:,1:,:].cpu().long()
        label2 = label.view(-1,1,1)
        label = torch.cat([label1,label2],1).view(-1,window_size)
        label = label.reshape(label.size(0)*label.size(1))
        output = model(seq)
        output = output.reshape(output.size(0)*output.size(1),-1)
        loss = criterion(output, label.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        writer.add_graph(model, seq)
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, current_epoch+num_epochs, train_loss / total_step))
    writer.add_scalar('第一阶段模型训练损失', train_loss / total_step, epoch + 1)
    metric(steps)
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))

loading data: 14it [00:48,  3.46s/it]
loading data: 5it [00:16,  3.25s/it]


elapsed_time: 64.804s
false positive (FP): 14131, false negative (FN): 0, Precision: 22.587%, Recall: 100.000%, F1-measure: 36.850%
Finished Predicting
正确率:22.781 准确率:22.587 召回率:100.0 F1值:36.85


loading data: 14it [00:55,  3.97s/it]
loading data: 5it [00:19,  3.97s/it]


elapsed_time: 75.506s
false positive (FP): 12845, false negative (FN): 54, Precision: 24.057%, Recall: 98.690%, F1-measure: 38.684%
Finished Predicting
正确率:29.514 准确率:24.057 召回率:98.69 F1值:38.684


loading data: 14it [00:54,  3.88s/it]
loading data: 5it [00:19,  3.83s/it]


...

loading data: 14it [00:45,  3.23s/it]
loading data: 5it [00:14,  2.94s/it]


elapsed_time: 60.318s
false positive (FP): 781, false negative (FN): 206, Precision: 83.376%, Recall: 95.004%, F1-measure: 88.811%
Finished Predicting
正确率:94.607 准确率:83.376 召回率:95.004 F1值:88.811


loading data: 0it [00:00, ?it/s]

Epoch [256/400], train_loss: 0.5358


loading data: 1it [00:04,  4.52s/it]

In [ ]:
steps

In [17]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + model_usl_name + '.pt')
print('Finished Training')

Finished Training


In [14]:
import unsupervised.predictor as unsupervised_predictor

In [16]:
re = unsupervised_predictor.predict(model, abnormal_dataloader,test_abnormal_session,10, window_size,ts=0.0005)

loading data: 5it [00:26,  5.33s/it]


In [1]:
from tensorboard.backend.event_processing import event_accumulator
from torch.utils.tensorboard import SummaryWriter
ea=event_accumulator.EventAccumulator('E:\
\python_workspace\\jupyter_notebook\\毕业设计\\dev\\end_to_end_dev_v0\\log\\unsupervise_lstm_SGD')
ea.Reload()
acc = ea.scalars.Items('正确率')
P = ea.scalars.Items('准确率')
R = ea.scalars.Items('召回率')
F1 = ea.scalars.Items('F1值')
Loss = ea.scalars.Items('第一阶段模型训练损失') 
process = {'accuracy':[[i.step,i.value] for i in acc]}
process['precision'] = [[i.step,i.value] for i in P]
process['recall'] = [[i.step,i.value] for i in R]
process['F1'] = [[i.step,i.value] for i in F1]
process['第一阶段模型训练损失'] = [[i.step,i.value] for i in Loss]

writer = SummaryWriter(log_dir='log/' + 'process')

for item in process['accuracy']:
    writer.add_scalar('正确率',item[1],item[0])
    
for item in process['precision']:
    writer.add_scalar('准确率',item[1],item[0])
    
for item in process['recall']:
    writer.add_scalar('召回率',item[1],item[0])
    
for item in process['F1']:
    writer.add_scalar('F1值',item[1],item[0])
    
f = open('unsupervised_process', 'wb')
pickle.dump(process, f)

with open('unsupervised_process', 'rb') as unsupervised_process:
    process = pickle.load(unsupervised_process)

In [ ]:
ea=event_accumulator.EventAccumulator('E:\
\python_workspace\\jupyter_notebook\\毕业设计\\dev\\end_to_end_dev_v0\\log\\unsupervise_lstm_SGD')
ea.Reload()
# print(ea.scalars.Keys())

In [40]:
acc = ea.scalars.Items('正确率')
P = ea.scalars.Items('准确率')
R = ea.scalars.Items('召回率')
F1 = ea.scalars.Items('F1值')
Loss = ea.scalars.Items('第一阶段模型训练损失') 

In [29]:
 ea.scalars.Items

<bound method Reservoir.Items of <tensorboard.backend.event_processing.reservoir.Reservoir object at 0x000002A9457267B8>>

In [41]:
process = {'accuracy':[[i.step,i.value] for i in acc]}
process['precision'] = [[i.step,i.value] for i in P]
process['recall'] = [[i.step,i.value] for i in R]
process['F1'] = [[i.step,i.value] for i in F1]
process['第一阶段模型训练损失'] = [[i.step,i.value] for i in Loss]

In [42]:
print(len(process['accuracy']))

265


In [34]:
writer = SummaryWriter(log_dir='log/' + 'process')

In [35]:
for item in process['accuracy']:
    writer.add_scalar('正确率',item[1],item[0])
    
for item in process['precision']:
    writer.add_scalar('准确率',item[1],item[0])
    
for item in process['recall']:
    writer.add_scalar('召回率',item[1],item[0])
    
for item in process['F1']:
    writer.add_scalar('F1值',item[1],item[0])

In [1]:
import pickle

In [ ]:
f = open('unsupervised_process', 'wb')
pickle.dump(process, f)

In [2]:
with open('unsupervised_process', 'rb') as unsupervised_process:
    process = pickle.load(unsupervised_process)

In [3]:
print(process)

{'accuracy': [[0, 22.781421661376953], [10, 29.513660430908203], [20, 30.147541046142578], [30, 30.147541046142578], [40, 56.612022399902344], [49, 56.612022399902344], [50, 56.612022399902344], [60, 56.612022399902344], [70, 56.612022399902344], [80, 68.49180603027344], [90, 94.95628356933594], [98, 94.95628356933594], [147, 94.95628356933594], [196, 94.95628356933594], [245, 94.95628356933594], [294, 95.00546264648438], [343, 94.98360443115234], [392, 94.96721649169922], [441, 94.9726791381836], [490, 94.98360443115234], [539, 95.0], [588, 95.0], [637, 95.0], [686, 95.01092529296875], [735, 95.02185821533203], [784, 95.02185821533203], [833, 95.02185821533203], [882, 94.98907470703125], [931, 95.03825378417969], [980, 95.04917907714844], [1029, 95.06011199951172], [1078, 95.04917907714844], [1127, 95.05464172363281], [1176, 95.06011199951172], [1225, 95.06011199951172], [1274, 95.06011199951172], [1323, 95.02185821533203], [1372, 95.0], [1421, 95.0], [1470, 94.90164184570312], [1519,

In [ ]:
for item in process['F1']:
    writer.add_scalar('F1值',item[1],item[0])

In [20]:
batch_size = 20000
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [21]:
normal_result = unsupervised_predictor.predict(model, dataloader,sessions,10, window_size,ts=0.0005)

loading data: 5it [00:24,  4.97s/it]


In [22]:
def show_result(result):
    TN = result[:3000].count(False)
    FN = result[3000:].count(False)
    print('TN:{} FN:{}'.format(TN,FN))
    print('Negative Precision:',TN/(TN+FN))
    TP = result[3000:].count(True)
    FP = result[:3000].count(True)
    print('TP:{} FP:{}'.format(TP,FP))
    print('Positive Precision:',TP/(TP+FP))

In [23]:
show_result(normal_result)

TN:2853 FN:31
Negative Precision: 0.989251040221914
TP:1968 FP:147
Positive Precision: 0.9304964539007092


In [24]:
abnormal_result = unsupervised_predictor.predict(model, dataloader,sessions,10, window_size,ts=0)

loading data: 5it [00:24,  4.82s/it]


In [68]:
show_result(abnormal_result)

TN:2938 FN:156
Negative Precision: 0.9495798319327731
TP:1843 FP:62
Positive Precision: 0.9674540682414698


In [39]:
f = open(lstm_dataset+predicted_normal_output,'w')
f.write('EventSequence\n')
for i,label in enumerate(normal_result):
    if label==False:
        f.write('"'+str(block_to_seqs[i][1])+'"\n')
f.close()

In [40]:
f = open(lstm_dataset+predicted_abnormal_output,'w')
f.write('EventSequence\n')
for i,label in enumerate(abnormal_result):
    if label:
        f.write('"'+str(block_to_seqs[i][1])+'"\n')
f.close()

In [41]:
input_size = 768

In [42]:
from utils.bert_utils import *

In [33]:
build_bert_cache('.\\data\\lstm\\dataset_official\\HDFS_templates.txt',"./data/lstm/bert_official_cache.pth")

In [36]:
predicted_normal_output

'unsupervised_normal_output.csv'

In [69]:
predicted_abnormal_output

'unsupervised_abnormal_output.csv'

In [43]:
train_dataset, test_dataset=generate_bert_data(lstm_dataset,predicted_normal_output,predicted_abnormal_output,"./data/lstm/bert_official_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 1895/1895 [00:45<00:00, 41.85it/s]


Number of sessions(.\data\lstm\dataset_official\): 4778
Number of normal sessions: 2883
Number of abnormal sessions: 1895


In [44]:
batch_size = 100
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [45]:
import supervised.model as supervised_model

In [46]:
model = supervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [50]:
model_sl_name = 'type={}_version={}'.format('official_supervised',version)

In [51]:
if os.path.exists(model_dir + '/' + model_sl_name + '.pt'):
    print('loading model {} success.'.format(model_sl_name))
    model.load_state_dict(torch.load(model_dir + '/' + model_name + '.pt'))

In [52]:
torch.cuda.empty_cache()

In [54]:
model.train()
supervised_model.train(model,train_dataloader,criterion,optimizer,current_epoch=0,num_epochs=30,input_size=input_size)

Epoch [1/30], train_loss: 0.1469
Epoch [2/30], train_loss: 0.1291
Epoch [3/30], train_loss: 0.1222
Epoch [4/30], train_loss: 0.1478
Epoch [5/30], train_loss: 0.1397
Epoch [6/30], train_loss: 0.1540
Epoch [7/30], train_loss: 0.1872
Epoch [8/30], train_loss: 0.2230
Epoch [9/30], train_loss: 0.1346
Epoch [10/30], train_loss: 0.1293
Epoch [11/30], train_loss: 0.1225
Epoch [12/30], train_loss: 0.1519
Epoch [13/30], train_loss: 0.1287
Epoch [14/30], train_loss: 0.1319
Epoch [15/30], train_loss: 0.1229
Epoch [16/30], train_loss: 0.1295
Epoch [17/30], train_loss: 0.1262
Epoch [18/30], train_loss: 0.1223
Epoch [19/30], train_loss: 0.1159
Epoch [20/30], train_loss: 0.1153
Epoch [21/30], train_loss: 0.1261
Epoch [22/30], train_loss: 0.1223
Epoch [23/30], train_loss: 0.1981
Epoch [24/30], train_loss: 0.1301
Epoch [25/30], train_loss: 0.1341
Epoch [26/30], train_loss: 0.1400
Epoch [27/30], train_loss: 0.1359
Epoch [28/30], train_loss: 0.1447
Epoch [29/30], train_loss: 0.1239
Epoch [30/30], train_lo

In [55]:
import numpy as np
def accuracy(y_pred, y_true):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    pred = np.argmax(y_pred.cpu().numpy(),1)
    for i,val in enumerate(pred):
        if val == 0:
            if y_true[i]==0:
                TN +=1
            else:
                FN +=1
        else:
            if y_true[i]==1:
                TP+=1
            else:
                FP+=1
    return TP,TN,FP,FN

In [56]:
def predict_result(model,data):
    model.eval()
    with torch.no_grad():
        epoch_loss = 0
        for step, (seq, label) in enumerate(data):
            seq = seq.clone().detach().view(-1, seq.shape[1], input_size).to(device)
            test_output = model(seq.to(device))
            if step == 0:
                output = test_output
                labels = label
            else:
                labels = torch.cat([labels, label], 0)
                output = torch.cat([output, test_output], 0)
            epoch_loss += criterion(test_output, label.to(device)).data
        TP,TN,FP,FN = accuracy(output, labels)
        epoch_accuracy = (TP+TN)/(FP+FN+TP+TN)
        epoch_loss = epoch_loss / len(data)
        print('TN:{} FN:{}'.format(TN,FN))
        print('Negative Precision:',round(TN/(TN+FN),3))
        print('TP:{} FP:{}'.format(TP,FP))
        print('Positive Precision:',round(TP/(TP+FP),3))
        print('loss: ', round(epoch_loss.item(), 3), 'accuracy: ', round(epoch_accuracy, 3))

In [70]:
predict_result(model,train_dataloader)

TN:1679 FN:33
Negative Precision: 0.981
TP:1108 FP:46
Positive Precision: 0.96
loss:  0.11 accuracy:  0.972


In [71]:
predict_result(model,test_dataloader)

TN:1137 FN:32
Negative Precision: 0.973
TP:722 FP:21
Positive Precision: 0.972
loss:  0.106 accuracy:  0.972


In [67]:
lstm_dataset

'.\\data\\lstm\\dataset_official\\'

In [59]:
supervised_input_normal = 'supervised_input_normal.csv'
supervised_input_abnormal = 'supervised_input_abnormal.csv'

In [82]:
test_dataset1,test_dataset2 =generate_bert_data(lstm_dataset,supervised_input_normal,supervised_input_abnormal,"./data/lstm/bert_official_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 2117/2117 [00:45<00:00, 46.64it/s]


Number of sessions(.\data\lstm\dataset_official\): 7111
Number of normal sessions: 4994
Number of abnormal sessions: 2117


In [76]:
train_dataset[:100][1]

tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 1])

In [81]:
torch.sum(test_dataset1[:][1])

tensor(1270)

In [83]:
test_dataloader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=True)
test_dataloader2 = DataLoader(test_dataset2, batch_size=batch_size)

In [84]:
predict_result(model,test_dataloader1)

TN:2790 FN:40
Negative Precision: 0.986
TP:1229 FP:207
Positive Precision: 0.856
loss:  0.196 accuracy:  0.942


In [85]:
predict_result(model,test_dataloader2)

TN:1898 FN:16
Negative Precision: 0.992
TP:832 FP:99
Positive Precision: 0.894
loss:  0.147 accuracy:  0.96


In [2]:
from unsupervised.wf_constructor import workflow_constructor,draw_wf


In [ ]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + model_usl_name + '.pt')
print('Finished Training')

In [3]:
wf = workflow_constructor(model,input_size=1)
wf.generate_seq([0],1,3)


NameError: name 'model' is not defined

In [ ]:
wf.workflow_construction([0],{30})
draw_wf(wf)